In [ ]:
%reload_ext autoreload
%autoreload 2


import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres, find_depth_neurons, common_utils, fit_gaussian_blob, size_control
from cottage_analysis.plotting import basic_vis_plots, plotting_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, close_loop_rsof, get_session_list, open_loop_rsof
from v1_depth_analysis.v1_manuscript_2023 import common_utils as v1_common_utils

In [ ]:
VERSION = 1
SAVE_ROOT = '/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver'+str(VERSION)+'/fig3/'
os.makedirs(SAVE_ROOT, exist_ok=True)

In [ ]:
# Concatenate all sessions
project='hey2_3d-vision_foodres_20220101'
flexilims_session = flz.get_flexilims_session(project)
results_all = v1_common_utils.concatenate_all_neurons_df(flexilims_session=flexilims_session,
                                                         session_list=get_session_list.get_all_sessions(project=project,mouse_list=[
                                                                                                                                    "PZAH6.4b",
                                                                                                                                    "PZAG3.4f",
                                                                                                                                    # "PZAH8.2h", 
                                                                                                                                    # "PZAH8.2i", 
                                                                                                                                    # "PZAH8.2f", 
                                                                                                                                    # "PZAH10.2d", 
                                                                                                                                    # "PZAH10.2f"
                                                                                                                                    ], 
                                                                                                        closedloop_only=False, 
                                                                                                        openloop_only=True),
                                                        #  ["PZAH6.4b_S20220419"],
                                                         filename="neurons_df.pickle",
                                                         cols=['roi','best_depth','preferred_depth_closedloop','preferred_depth_closedloop_crossval',
                                                               'depth_tuning_test_rsq_closedloop','depth_tuning_test_spearmanr_pval_closedloop',
                                                               'preferred_RS_closedloop_g2d','preferred_RS_closedloop_crossval_g2d',
                                                               'preferred_OF_closedloop_g2d','preferred_OF_closedloop_crossval_g2d',
                                                               'rsof_rsq_closedloop_crossval_g2d','rsof_rsq_closedloop_g2d',
                                                               'preferred_RS_openloop_actual_g2d','preferred_OF_openloop_actual_g2d',
                                                               'rsof_rsq_openloop_actual_g2d',],
                                                         
                                                         read_iscell=True,
                                                         verbose=True)
results_all["preferred_RS_closedloop_g2d"] = results_all["preferred_RS_closedloop_g2d"]*100
results_all["preferred_depth_closedloop"] = results_all["preferred_depth_closedloop"]*100
results_all["preferred_OF_closedloop_g2d"] = np.degrees(results_all["preferred_OF_closedloop_g2d"])
results_all["preferred_RS_openloop_actual_g2d"] = results_all["preferred_RS_openloop_actual_g2d"]*100
results_all["preferred_OF_openloop_actual_g2d"] = np.degrees(results_all["preferred_OF_openloop_actual_g2d"])


In [ ]:
fig=plt.figure()
open_loop_rsof.plot_speeds_scatter(
    fig=fig,
    neurons_df=results_all,
    xcol="preferred_RS_closedloop_g2d",
    ycol="preferred_RS_openloop_actual_g2d",
    xlabel="Running speed\nclosedloop(cm/s)",
    ylabel="Running speed\nopenloop (cm/s)",
    s=10,
    alpha=0.2,
    c='g',
    plot_x=0,
    plot_y=0.3,
    plot_width=0.15,
    plot_height=0.2,
    cbar_width=0.01,
    fontsize_dict={"title": 15, "label": 10, "tick": 10},)

open_loop_rsof.plot_speeds_scatter(
    fig=fig,
    neurons_df=results_all,
    xcol="preferred_OF_closedloop_g2d",
    ycol="preferred_OF_openloop_actual_g2d",
    xlabel="Optic flow speed\nclosedloop(cm/s)",
    ylabel="Optic flow speed\nopenloop (cm/s)",
    s=10,
    alpha=0.2,
    c='b',
    plot_x=0.4,
    plot_y=0.3,
    plot_width=0.15,
    plot_height=0.2,
    cbar_width=0.01,
    fontsize_dict={"title": 15, "label": 10, "tick": 10},)

# decoder plots

# example open loop matrix, corner of RS 0 and OF 0

In [ ]:

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name="PZAG3.4f_S20220526", flexilims_session=flexilims_session, project=None, conflicts="skip"
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

In [ ]:
neurons_df_example.columns